In [1]:
# so notebook can access the folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch torchaudio
!pip install datasets
!pip install pandas
!pip install transformers
!pip install evaluate
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00


In [3]:
!tar -xzvf /content/drive/MyDrive/DS565_Project/data/audio.tar.gz

Streaming output truncated to the last 5000 lines.
audio/train/Ses03M_impro02_F027.wav
audio/train/Ses03M_script03_1_F009.wav
audio/train/Ses01M_impro07_F009.wav
audio/train/Ses03M_impro02_F030.wav
audio/train/Ses04M_script03_2_M038.wav
audio/train/Ses02F_script03_2_F004.wav
audio/train/Ses04M_impro08_M006.wav
audio/train/Ses04F_impro02_M018.wav
audio/train/Ses03F_script02_1_F012.wav
audio/train/Ses04F_script02_2_F018.wav
audio/train/Ses02M_impro05_F018.wav
audio/train/Ses04F_impro05_F014.wav
audio/train/Ses03M_impro05b_M020.wav
audio/train/Ses02M_script01_1_M033.wav
audio/train/Ses05M_script02_1_F001.wav
audio/train/Ses01F_impro06_F002.wav
audio/train/Ses05M_impro04_M014.wav
audio/train/Ses02F_script01_1_F041.wav
audio/train/Ses05F_script02_2_M005.wav
audio/train/Ses04F_script01_1_M015.wav
audio/train/Ses02F_script02_1_F020.wav
audio/train/Ses02M_impro02_M018.wav
audio/train/Ses01F_script03_1_M017.wav
audio/train/Ses01M_impro06_F018.wav
audio/train/Ses01M_impro03_M002.wav
audio/train/

# Load dataset as HuggingFace Dataset

In [46]:
import os
import json
import pandas as pd
data_folder = 'audio'

with open(os.path.join(data_folder, 'metadata.json'), 'r') as f_in:
  metadata = json.load(f_in)

train = pd.read_csv(os.path.join(data_folder, 'train.csv'))
val = pd.read_csv(os.path.join(data_folder, 'val.csv'))
test = pd.read_csv(os.path.join(data_folder, 'test.csv'))

print(
    len(metadata),
    len(train),
    len(val),
    len(test)
)

assert len(metadata) == len(train) + len(val) + len(test)
train.head()

10039 6023 2008 2008


,name,emotion
0,Ses02M_impro08_M013,6
1,Ses01M_impro02_M012,8
2,Ses04F_script03_2_M020,10
3,Ses04F_script01_1_F042,0
4,Ses05M_script03_2_M016,4


In [59]:
import os
import json
import pandas as pd
data_folder = 'audio'

train = pd.read_csv(os.path.join(data_folder, 'train.csv')).rename(columns = {'emotion':'label'})
val = pd.read_csv(os.path.join(data_folder, 'val.csv')).rename(columns = {'emotion':'label'})
test = pd.read_csv(os.path.join(data_folder, 'test.csv')).rename(columns = {'emotion':'label'})

train['file_name'] = train['name'] + '.wav'
val['file_name'] = val['name'] + '.wav'
test['file_name'] = test['name'] + '.wav'

train[['file_name', 'label']].to_csv('audio/train/metadata.csv', index=False)
val[['file_name', 'label']].to_csv('audio/valid/metadata.csv',index=False)
test[['file_name', 'label']].to_csv('audio/test/metadata.csv',index=False)

In [60]:
# shape: (timestamp, label, session, improv)
metadata['Ses04F_script03_2_M020']

['[100.8400 - 107.2200]', 'xxx', 'Session4', 'Ses04F_script03_2']

Load it as a huggingface dataset

In [61]:
dataset = load_dataset("audiofolder", data_dir='audio')
dataset

Resolving data files:   0%|          | 0/6024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2009 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2009 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 6023
    })
    validation: Dataset({
        features: ['audio', 'label'],
        num_rows: 2008
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 2008
    })
})

# Wav2Vec2-BERT

> https://huggingface.co/docs/transformers/en/model_doc/wav2vec2-bert#transformers.Wav2Vec2BertModel

In [65]:
from transformers import AutoProcessor, Wav2Vec2BertModel
import torch
from datasets import load_dataset

train = dataset['train']
sampling_rate = train.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")
model = Wav2Vec2BertModel.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")

# audio file is decoded on the fly
inputs = processor(train[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/299 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[1, 49, 1024]